In [1]:
%matplotlib inline
import matplotlib
matplotlib.style.use('bmh')
import paramselect
import os
import fnmatch
def recursive_glob(start, pattern):
    matches = []
    for root, dirnames, filenames in os.walk(start):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))
    return matches
comps = ['AL', 'NI', 'VA']
#phase = 'FCC_L12'
#configuration = ["NI", "NI", "NI", "AL", "VA"]
#ratios = [0.25, 0.25, 0.25, 0.25, 1]
phase = 'BCC_B2'
ratios = [0.5, 0.5, 1]
configuration = ["AL", ["AL", "NI"], "VA"]
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))

In [2]:
parameters = paramselect.fit_formation_energy(comps, phase, configuration, datasets)
print(parameters)

[{u'phases': [u'BCC_B2'], u'values': array([[[-19504.41974, -41109.66717]]]), u'components': [u'AL', u'NI', u'VA'], u'reference': u'Chao Jiang 2009 SQS calculations', u'solver': {u'sublattice_occupancies': [[1, [0.5, 0.5], 1], [1, [0.6, 0.4], 1], [1, [0.75, 0.25], 1], [1, [0.5, 0.5], 1], [1, [0.5, 0.5], 1], [1, [0.25, 0.75], 1], [1, [0.6, 0.4], 1], [1, [0.5, 0.5], 1]], u'comment': u'BCC_B2 sublattice configuration (2SL)', u'sublattice_configurations': [[u'AL', [u'NI', u'VA'], u'VA'], [u'AL', [u'NI', u'VA'], u'VA'], [u'NI', [u'AL', u'NI'], u'VA'], [u'NI', [u'AL', u'NI'], u'VA'], [u'AL', [u'AL', u'NI'], u'VA'], [u'AL', [u'AL', u'NI'], u'VA'], [u'NI', [u'AL', u'VA'], u'VA'], [u'NI', [u'AL', u'VA'], u'VA']], u'mode': u'manual', u'sublattice_site_ratios': [0.5, 0.5, 1]}, u'output': u'HM_FORM', u'conditions': {u'P': 101325, u'T': 300}}]
[[ 1.]]
[-41109.66717]
((1,), 'rss:', 0.0, 'AIC:', -inf)
{1: -41109.667170000001, T*log(T): 0, T: 0, T**3: 0, 1/T: 0, T**2: 0}


In [3]:
paramselect.plot_parameters(comps, phase, configuration, ratios, datasets, parameters)

NotImplementedError: Plotting non-endmembers is not supported yet